In [1]:
import scipy.signal
import sys
import torch
import torch.nn as nn
import numpy as np

In [2]:
from typing import Dict, List, Optional, Tuple
import gym
from PIL import Image
# from pyvirtualdisplay import Display
# Display().start()
from datetime import datetime
from tqdm import tqdm

In [3]:
import math
import random
from copy import deepcopy
import torch
from torch.optim import Adam
from torch.optim import RMSprop
import gym
import time
from collections import namedtuple, deque
import neptune.new as neptune

In [4]:
import robosuite as suite
from robosuite.controllers import load_controller_config
from robosuite.controllers.controller_factory import reset_controllers
from robosuite.utils import observables
from robosuite.utils.input_utils import *
from robosuite.robots import Bimanual
import imageio
import numpy as np
import robosuite.utils.macros as macros
macros.IMAGE_CONVENTION = "opencv"

In [5]:
nep_log = neptune.init(
    project="xhnfirst/DDPG-robosuite",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NTg5MDI2OS01MTVmLTQ2YjUtODA1Yy02ZWQyNDgxZDcwN2UifQ==",
)

https://app.neptune.ai/xhnfirst/DDPG-robosuite/e/DDPGROB-227
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [6]:
options = {
    'env_name': 'EElab_test5',
    "robots": "UR5e"
}
controller_name = "JOINT_VELOCITY"
options["controller_configs"] = suite.load_controller_config(default_controller=controller_name)

env = suite.make(
    **options,
    has_renderer=False,
    has_offscreen_renderer=True,
    ignore_done=True,
    use_camera_obs=False,
    gripper_types=None,
    renderer = 'mujoco',

)

test_env = suite.make(
    **options,
    has_renderer=False,
    has_offscreen_renderer=False,
    ignore_done=True,
    use_camera_obs=False,
    gripper_types=None,
    renderer = 'mujoco',
)


video_env = suite.make(
    **options,
    gripper_types=None,
    has_renderer=False,
    has_offscreen_renderer=True,
    ignore_done=True,
    use_camera_obs=True,
    use_object_obs=True, 
    camera_names='Labviewer',
    camera_heights=512,
    camera_widths=512,
    # control_freq=200,
    renderer = 'mujoco',
)

frame = []
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device = ', device)

device =  cuda


In [7]:
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


class MLPActor(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation, act_limit):
        super().__init__()
        pi_sizes = [obs_dim] + list(hidden_sizes) + [act_dim]
        self.pi = mlp(pi_sizes, activation, nn.Tanh)
        self.act_limit = act_limit

    def forward(self, obs):
        # Return output from network scaled to action space limits.
        return self.act_limit * self.pi(obs)

class MLPQFunction(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.q = mlp([obs_dim + act_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs, act):
        q = self.q(torch.cat([obs, act], dim=-1))
        return torch.squeeze(q, -1) # Critical to ensure q has right shape.

class MLPActorCritic(nn.Module):

    def __init__(self, hidden_sizes=(256,256),
                 activation=nn.ReLU, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super().__init__()

        obs_dim = 35
        act_dim = 6
        act_limit = 1

        # build policy and value functions
        self.pi = MLPActor(obs_dim, act_dim, hidden_sizes, activation, act_limit).to(device)
        self.q = MLPQFunction(obs_dim, act_dim, hidden_sizes, activation).to(device)

    def act(self, obs):
        with torch.no_grad():
            return self.pi(obs)

In [8]:
Transition = namedtuple('Transition',
                        ('obs', 'act', 'rew', 'next_obs', 'done'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [9]:

params = {
    "dropout": 0.2,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "hid": 256,
    "l": 3,
    "seed": 0,
    "steps_per_epoch": 3000,
    "steps_video": 30000,
    "epochs": 1000,
    "replay_size": int(1e8),
    "gamma": 0.99,
    "polyak": 0.995,
    "pi_lr": 1e-4,
    "q_lr": 1e-4,
    "batch_size": 1000,
    "start_steps": 10000, 
    "update_after": 5000,
    "update_every": 100,
    "act_noise": 0.01,
    "num_test_episodes": 5,
    "max_ep_len": 300,
    "max_video_len": 300,
    "save_model_len": 10000,
    # "obs_dim": 47,
    # "act_dim": 7,
    # "act_limit": 1
}

ac_kwargs=dict(hidden_sizes=[params["hid"]]*params["l"])

In [10]:
nep_log["parameters"] = params

torch.manual_seed(params["seed"])
np.random.seed(params["seed"])

obs_dim = 35
print('obs_dim = ', obs_dim)
act_dim = 6
print('act_dim = ', act_dim)
# Action limit for clamping: critically, assumes all dimensions share the same bound!
act_limit = 1
print('act_limit = ', act_limit)
# Create actor-critic module and target networks
ac = MLPActorCritic(**ac_kwargs)
ac_targ = deepcopy(ac)

# Freeze target networks with respect to optimizers (only update via polyak averaging)
for p in ac_targ.parameters():
    p.requires_grad = False

memory = ReplayMemory(params["replay_size"])

obs_dim =  35
act_dim =  6
act_limit =  1


In [11]:
# Set up function for computing DDPG Q-loss
def compute_loss_q(data):

    o = torch.cat(data.obs).float()
    a = torch.cat(data.act).float()
    r = torch.cat(data.rew).float()
    o2 =torch.cat(data.next_obs).float()
    d = torch.cat(data.done).float()

    q = ac.q(o,a)


    # Bellman backup for Q function
    with torch.no_grad():
        q_pi_targ = ac_targ.q(o2, ac_targ.pi(o2))
        backup = r + params["gamma"] * (1 - d) * q_pi_targ

    # MSE loss against Bellman backup
    loss_q = ((q - backup)**2).mean()

    return loss_q

# Set up function for computing DDPG pi loss
def compute_loss_pi(data):

    o = torch.cat(data.obs).float()

    q_pi = ac.q(o, ac.pi(o))

    return -q_pi.mean()


In [12]:
pi_optimizer = RMSprop(ac.pi.parameters(), lr=params["pi_lr"])
q_optimizer = RMSprop(ac.q.parameters(), lr=params["q_lr"])

def update(data):
    # First run one gradient descent step for Q.


    q_optimizer.zero_grad()
    loss_q = compute_loss_q(data)

    loss_q.backward()

    q_optimizer.step()


    # Freeze Q-network so you don't waste computational effort 
    # computing gradients for it during the policy learning step.
    for p in ac.q.parameters():
        p.requires_grad = False

    # Next run one gradient descent step for pi.
    pi_optimizer.zero_grad()
    loss_pi = compute_loss_pi(data)
    loss_pi.backward()
    pi_optimizer.step()

    # Unfreeze Q-network so you can optimize it at next DDPG step.
    for p in ac.q.parameters():
        p.requires_grad = True



    # Finally, update target networks by polyak averaging.
    with torch.no_grad():
        for p, p_targ in zip(ac.parameters(), ac_targ.parameters()):
            # NB: We use an in-place operations "mul_", "add_" to update target
            # params, as opposed to "mul" and "add", which would make new tensors.
            p_targ.data.mul_(params["polyak"])
            p_targ.data.add_((1 - params["polyak"]) * p.data)


In [13]:



def get_action(o, noise_scale):
    a = ac.act(torch.as_tensor(o, dtype=torch.float32))
    a += noise_scale * torch.randn(act_dim).to(device)
    return torch.clip(a, -act_limit, act_limit)

def test_agent(epoch):
    test_main = 0
    test_step = 0
    for j in range(params["num_test_episodes"]):
        obs, d, test_ep_ret, test_ep_len = test_env.reset(), False, 0, 0
        o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
        o = torch.tensor([o], dtype=torch.float32, device=device)
        while not(d or (test_ep_len == params["max_ep_len"])):
            a_cpu = get_action(o, 0).cpu().data.numpy()
            obs, r, d, _ = test_env.step(a_cpu[0])
            o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
            o = torch.tensor([o], dtype=torch.float32, device=device)
            test_ep_ret += r
            test_ep_len += 1
        test_ep_main = test_ep_ret/test_ep_len
        test_step +=1
        test_main += test_ep_main
    print('test_rew_main = ', float(test_main/test_step))
    nep_log["test/reward"].log(test_main/test_step)
    
def video_agent(epoch):
    obs, d, test_ep_len = video_env.reset(), False, 0
    o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
    o = torch.tensor([o], dtype=torch.float32, device=device)
    now = datetime.now()
    current_time = str(now.isoformat())
    writer = imageio.get_writer(
        "/home/xhnfly/Cosmic_rays_X/X_Robot/robosuite/robosuite/demos/video/DDPG_re_touch_2_15_300/DDPG_UR5_%s_ep_%d.mp4" % (current_time, epoch), fps=100)
    frame = obs["Labviewer_image"]
    writer.append_data(frame)

    while not(d or (test_ep_len == params["max_video_len"])):
        a_cpu = get_action(o, 0).cpu().data.numpy()
        obs, _, d, _ = video_env.step(a_cpu[0])
        o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
        o = torch.tensor([o], dtype=torch.float32, device=device)
        frame = obs["Labviewer_image"]
        writer.append_data(frame)
        test_ep_len += 1
    writer.close()
    nep_log['video'] = neptune.types.File('/home/xhnfly/Cosmic_rays_X/X_Robot/robosuite/robosuite/demos/video/DDPG_re_touch_2_15_300/DDPG_UR5_%s_ep_%d.mp4' % (current_time, epoch))





In [14]:
# obs = {
#     'robot0_joint_pos_cos': None,
#     'robot0_joint_pos_sin': None,
#     'robot0_joint_vel': None,
#     'robot0_eef_pos': None,
#     'robot0_eef_quat': None,
#     'robot0_gripper_qpos': None,
#     'robot0_gripper_qvel': None,
#     'cubeA_pos': None,
#     'cubeA_quat': None,
#     'cubeB_pos': None,
#     'cubeB_quat': None,
#     'gripper_to_cubeA': None,
#     'gripper_to_cubeB': None,
#     'cubeA_to_cubeB': None,
# }

obs, ep_ret, ep_len = env.reset(), 0, 0

o = list(obs['robot0_proprio-state']) + list(obs['object-state'])

# env.viewer.set_camera(camera_id=0)


# Define neutral value
neutral = np.zeros(7)

# Keep track of done variable to know when to break loop

# Prepare for interaction with environment
total_steps = params["steps_per_epoch"] * params["epochs"]
start_time = time.time()

o = torch.tensor([o], device=device)


start_time_rec = datetime.now()
r_true = 0
total_main = 0
ep_rew_main = 0
reward_dict={}

In [15]:
# Main loop: collect experience in env and update/log each epoch
low, high = env.action_spec

for t in tqdm(range(total_steps)):
    
    # Until start_steps have elapsed, randomly sample actions
    # from a uniform distribution for better exploration. Afterwards, 
    # use the learned policy (with some noise, via act_noise). 
    if t > params["start_steps"]:
        a = get_action(o, params["act_noise"])      # Tensor
    else:
        a = torch.tensor([np.random.uniform(low, high)], dtype=torch.float32, device=device)
        
    a_cpu = a.cpu().data.numpy()
    # Step the env
    obs2, r, d, _ = env.step(a_cpu[0])
    
    o2 = list(obs2['robot0_proprio-state']) + list(obs2['object-state'])
    # print('len(o2) = ', len(o2))

    ep_len += 1
    total_main += r


    # Ignore the "done" signal if it comes from hitting the time
    # horizon (that is, when it's an artificial terminal signal
    # that isn't based on the agent's state)
    d = False if ep_len==params["max_ep_len"] else d

    o2 = torch.tensor([o2], dtype=torch.float32, device=device)
    r = torch.tensor([r], dtype=torch.float32, device=device)
    d = torch.tensor([d], dtype=torch.float32, device=device)

    # Store experience to replay buffer
    memory.push(o, a, r, o2, d)
    nep_log["train/o"].log(o)
    nep_log["train/a"].log(a)
    nep_log["train/r"].log(r)
    nep_log["train/o2"].log(o2)
    nep_log["train/d"].log(d)

    # Super critical, easy to overlook step: make sure to update 
    # most recent observation!
    o=o2
    ep_ret += r
    
    
    # End of trajectory handling
    if d or (ep_len == params["max_ep_len"]):
        ep_rew = ep_ret/ep_len
        ep_rew_main += ep_rew
        obs, ep_ret, ep_len = env.reset(), 0, 0
        o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
        o = torch.tensor([o], device=device)


    # Update handling
    if t >= params["update_after"] and t % params["update_every"] == 0:
        for i in range(params["update_every"]):

            transitions = memory.sample(params["batch_size"])
            # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
            # detailed explanation). This converts batch-array of Transitions
            # to Transition of batch-arrays.
            batch = Transition(*zip(*transitions))
            update(data=batch)

    # End of epoch handling
    if (t+1) % params["steps_per_epoch"] == 0:
        epoch = (t+1) // params["steps_per_epoch"]
        train_reward = ep_rew_main/epoch
        nep_log["train/reward"].log(train_reward)
        print('ep_rew_main = ', train_reward.cpu().data.numpy())
        ep_rew_main = 0
        # Test the performance of the deterministic version of the agent.
        test_agent(epoch)
        

    if (t+1) % params["steps_video"] == 0:
        epoch = (t+1) // params["steps_per_epoch"]
        now = datetime.now()
        current_time = str(now.isoformat())
        print('current_time = ', current_time)
        video_agent(epoch)
        now = datetime.now()
        current_time = str(now.isoformat())
        print('current_time = ', current_time)

    if (t+1) % params["save_model_len"] == 0:
        epoch = (t+1) // params["steps_per_epoch"]
        now = datetime.now()
        current_time = str(now.isoformat())
        torch.save({
                    'model of ac.q': ac.q.state_dict(),
                    'model of ac.pi': ac.pi.state_dict(),
                    'q_optimizer_state_dict': q_optimizer.state_dict(),
                    'pi_optimizer_state_dict': pi_optimizer.state_dict(),
                    
                    }, "model_nn/touch/model_nn_%s%d.pt" % (current_time, epoch))


        

  0%|          | 0/3000000 [00:00<?, ?it/s]/tmp/ipykernel_15790/1915781525.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180588308/work/torch/csrc/utils/tensor_new.cpp:201.)
  a = torch.tensor([np.random.uniform(low, high)], dtype=torch.float32, device=device)
  0%|          | 2980/3000000 [00:21<4:02:37, 205.87it/s]

ep_rew_main =  [1.3976852e-05]


  0%|          | 3021/3000000 [00:29<67:50:31, 12.27it/s]

test_rew_main =  7.0645495580884026e-06


  0%|          | 5996/3000000 [01:09<8:59:52, 92.43it/s] 

...

 23%|██▎       | 693700/3000000 [8:08:46<27:04:59, 23.65it/s] 


KeyboardInterrupt: 

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.


All 1 operations synced, thanks for waiting!


In [ ]:
model = ac.q
print("Model_q's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

model = ac.pi
print("Model_pi's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
print("pi_optimizer's state_dict:")
for var_name in pi_optimizer.state_dict():
    print(var_name, "\t", pi_optimizer.state_dict()[var_name])

print("q_optimizer's state_dict:")
for var_name in q_optimizer.state_dict():
    print(var_name, "\t", q_optimizer.state_dict()[var_name])

In [ ]:
now = datetime.now()

current_time = str(now.isoformat())



torch.save({
            'model of ac.q': ac.q.state_dict(),
            'model of ac.pi': ac.pi.state_dict(),
            'q_optimizer_state_dict': q_optimizer.state_dict(),
            'pi_optimizer_state_dict': pi_optimizer.state_dict(),
            
            }, "model_nn/model_nn_%s.pt" % current_time)



In [ ]:
nep_log.stop()